In [1]:
Pkg.checkout("LowRankModels")
using LowRankModels, Plots

INFO: Checking out LowRankModels master...
INFO: Pulling LowRankModels latest master...
INFO: No packages to install, update or remove


LowRankModels.jl is a julia package for modeling and fitting generalized low rank models (GLRMs). GLRMs model a data array by a low rank matrix, and include many well known models in data analysis, such as principal components analysis (PCA), matrix completion, robust PCA, nonnegative matrix factorization, k-means, and many more.

LowRankModels.jl makes it easy to mix and match loss functions and regularizers to construct a model suitable for a particular data set. In particular, it supports

   * using different loss functions for different columns of the data array, which is useful when data types are heterogeneous (eg, real, boolean, and ordinal columns);
   * fitting the model to only some of the entries in the table, which is useful for data tables with many missing (unobserved) entries; and
   * adding offsets and scalings to the model without destroying sparsity, which is useful when the data is poorly scaled.


# Losses

Supported losses include:
   * quadratic loss - QuadLoss()
   * hinge loss - HingeLoss()
   * logistic loss - LogisticLoss()
   * poisson loss - PoissonLoss()
   * weighted hinge loss - WeightedHingeLoss()
   * l1 loss - L1Loss()
   * ordinal hinge loss - OrdinalHingeLoss()
   * periodic loss - PeriodicLoss()
   * multinomial categorical loss - MultinomialLoss()
   * multinomial ordinal (aka ordered logit) loss - OrderedMultinomialLoss()

In [2]:
# loss function
loss = QuadLoss()

LoadError: UndefVarError: QuadLoss not defined

In [3]:
# the quad loss returns the sum of square differences between its first and second argument
evaluate(loss, 2., 3.)

LoadError: UndefVarError: evaluate not defined

In [4]:
# can scale the loss by any factor
evaluate(3*loss, 2., 3.)

LoadError: UndefVarError: evaluate not defined

In [5]:
evaluate(2*loss, [1., 1.], [3., 3.])

LoadError: UndefVarError: evaluate not defined

In [6]:
# can also evaluate the gradient wrt the first argument
grad(loss, 2., 0.)

LoadError: UndefVarError: grad not defined

In [7]:
grad(loss, [2., 0.], [0., 2.])

LoadError: UndefVarError: grad not defined

In [8]:
grad(L1Loss(), [2., 0.], [0., 2.])

LoadError: UndefVarError: grad not defined

# Regularizer

Supported regularizers include:

   * quadratic regularization - QuadReg()
   * constrained squared euclidean norm - QuadConstraint()
   * l1 regularization - OneReg()
   * no regularization - ZeroReg()
   * nonnegative constraint - NonNegConstraint() (eg, for nonnegative matrix factorization)
   * 1-sparse constraint - OneSparseConstraint() (eg, for orthogonal NNMF)
   * unit 1-sparse constraint - UnitOneSparseConstraint() (eg, for k-means)
   * simplex constraint - SimplexConstraint()
   * l1 regularization, combined with nonnegative constraint - NonNegOneReg()
   * fix features at values y0 - FixedLatentFeaturesConstraint(y0)

In [9]:
# regularizers
lambda = 1

nonneg = NonNegConstraint()
l1 = OneReg(lambda)
l2 = QuadReg(lambda)

LoadError: UndefVarError: NonNegConstraint not defined

In [10]:
# can evaluate the proximal operator of the regularizer
prox(nonneg, -1)

LoadError: UndefVarError: prox not defined

In [11]:
prox(nonneg, 10)

LoadError: UndefVarError: prox not defined

In [12]:
prox(nonneg, [-5, -1, 0, 1, 5])

LoadError: UndefVarError: prox not defined

In [13]:
# can evaluate the proximal operator of lambda times the regularizer
λ = .01
prox(l1, 1, λ)

LoadError: UndefVarError: prox not defined

In [14]:
# chain rule: 
# gradient of ||Xw - y||^2 wrt w is X' * <gradient of ||z-y||^2 wrt z>, 
# where z = X*w

We can use these to easily run proximal gradient on any combination of loss function and regularizers.

In [15]:
import LowRankModels: evaluate, grad
evaluate(loss::Loss, X::Array{Float64,2}, w, y) = evaluate(loss, X*w, y)
grad(loss::Loss, X::Array{Float64,2}, w, y) = X'*grad(loss, X*w, y)

INFO: Precompiling module LowRankModels.
ERROR: LoadError: LoadError: syntax: invalid operator ".!"
 in include_from_node1(::String) at ./loading.jl:488
 in include_from_node1(::String) at /Applications/Julia-0.5.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 in include_from_node1(::String) at ./loading.jl:488
 in include_from_node1(::String) at /Applications/Julia-0.5.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 in macro expansion; at ./none:2 [inlined]
 in anonymous at ./<missing>:?
 in eval(::Module, ::Any) at ./boot.jl:234
 in eval(::Module, ::Any) at /Applications/Julia-0.5.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 in process_options(::Base.JLOptions) at ./client.jl:239
 in _start() at ./client.jl:318
 in _start() at /Applications/Julia-0.5.app/Contents/Resources/julia/lib/julia/sys.dylib:?
while loading /Users/madeleine/.julia/v0.5/LowRankModels/src/scikitlearn.jl, in expression starting on line 56
while loading /Users/madeleine/.julia/v0.5/LowRankModels/src

LoadError: Failed to precompile LowRankModels to /Users/madeleine/.julia/lib/v0.5/LowRankModels.ji.

In [16]:
# proximal gradient method
function proxgrad(loss::Loss, reg::Regularizer, X, y;
                  maxiters::Int = 10, stepsize::Number = 1., 
                  ch::ConvergenceHistory = ConvergenceHistory("proxgrad"))
    w = zeros(size(X,2))
    for t=1:maxiters
        t0 = time()
        # gradient step
        g = grad(loss, X, w, y)
        w = w - stepsize*g
        # prox step
        w = prox(reg, w, stepsize)
        # record objective value
        update_ch!(ch, time() - t0, obj = evaluate(loss, X, w, y) + evaluate(reg, w))
    end
    return w
end

LoadError: UndefVarError: Loss not defined

In [17]:
srand(0)
X, y = rand(6,3), rand(6);
ch = ConvergenceHistory("NNLS")
w = proxgrad(QuadLoss(), NonNegConstraint(), X, y; 
             stepsize=.1, maxiters=50,
             ch = ch)

plot(ch.objective)
xlabel("iteration")
ylabel("objective")

LoadError: UndefVarError: ConvergenceHistory not defined

In [18]:
semilogy(ch.objective - ch.objective[end])
xlabel("iteration")
ylabel("objective")

LoadError: UndefVarError: ch not defined

# Generalized Low Rank Models

GLRMs form a low rank model for tabular data A with m rows and n columns, which can be input as an array or any array-like object (for example, a data frame). It is fine if only some of the entries have been observed (i.e., the others are missing or NA); the GLRM will only be fit on the observed entries $\Omega$.

The desired model is specified by choosing a rank k for the model, an array of loss functions losses, and two regularizers, $r_x$ and $r_w$. The data is modeled as $X^TW$, where $X$ is a $k\times m$ matrix and $W$ is a $k\times n$ matrix. $X$ and $W$ are found by solving the optimization problem

$$\min \sum_{(i,j) \in \Omega} loss_j\bigg((X^TW)[i,j], Y[i,j]\bigg) + \sum_i r_x(X[:,i]) + \sum_j r_y(W[:,j])$$

To form a GLRM, the user specifies

   * the data $Y$ (any AbstractArray, such as an array, a sparse matrix, or a data frame)
   * the array of loss functions losses
   * the regularizers $r_x$ and $r_w$
   * the rank $k$
   * the observations $\Omega$


In [19]:
# example
Y = randn(10, 10)
loss = QuadLoss()
nonneg = NonNegConstraint()
k = 5
Ω = [(rand(1:10), rand(1:10)) for iobs in 1:50] # observe 50 random entries, with replacement
glrm = GLRM(Y, loss, nonneg, nonneg, k, obs=Ω);

LoadError: UndefVarError: QuadLoss not defined

In [20]:
# To fit the model, call
X,W,ch = fit!(glrm)

LoadError: UndefVarError: fit! not defined

This runs an alternating directions proximal gradient method on glrm to find the $X$ and $W$ minimizing the objective function.

In [21]:
X

6×3 Array{Float64,2}:
 0.823648  0.0423017  0.260036
 0.910357  0.0682693  0.910047
 0.164566  0.361828   0.167036
 0.177329  0.973216   0.655448
 0.27888   0.585812   0.575887
 0.203477  0.539289   0.868279

In [22]:
W

LoadError: UndefVarError: W not defined

In [23]:
plot(ch.objective)
xlabel("iteration")
ylabel("objective")

LoadError: UndefVarError: plot not defined

The losses argument can also be an array of loss functions, with one for each column (in order). For example, for a data set with 3 columns, you could use:

In [24]:
losses = Loss[QuadLoss(), LogisticLoss(), HingeLoss()]

LoadError: UndefVarError: Loss not defined

Similiarly, the $r_w$ argument can be an array of regularizers, with one for each column (in order). For example, for a data set with 3 columns, you could use:

In [25]:
rw = Regularizer[QuadReg(1), QuadReg(10), OneReg()]

LoadError: UndefVarError: Regularizer not defined

Example: PCA

In [26]:
# minimize ||Y - XW||^2
function fit_pca(m,n,k)
	# matrix to encode
	Y = randn(m,k)*randn(k,n)
	loss = QuadLoss()
	r = ZeroReg()
	glrm = GLRM(Y,loss,r,r,k)
	X,W,ch = fit!(glrm)
	println("Convergence history:",ch.objective)
	return Y,X,W,ch
end

fit_pca (generic function with 1 method)